<center>
<h1>Franc carreau</h1>
</center>

<h2>Présentation</h2>

Dans l'article Carreau de l’Encyclopédie de Diderot et d’Alembert, on peut lire:

<i>Franc-Carreau, sorte de jeu dont M. de Buffon a donné le calcul en 1733, avant que d'être de
l'Académie des Sciences. Voici l'extrait qu'on trouve de son mémoire sur ce sujet, dans le
volume de l'Académie pour cette année-là.
Dans une chambre carrelée de carreaux égaux, & supposés réguliers, on jette en l'air un
louis ou un écu, & on demande combien il y a à parier que la pièce ne tombera que sur un
seul carreau, ou franchement.</i>

On considère un carrelage de carré de côtés 1 et un disque de rayon $r<\frac12$.

Calculer la probabilité que le disque ne coupe aucun bord du carrelage carré.

Effectuer 100 000 simulations.

<h2>Solution</h2>

Pour qu'un disque ne coupe pas le carré de côté 1, il faut que son centre soit dans le carré de côté $(1-2.r)$. Exemple: disque $A$; à la limite disque $C$. Contre-exemple: disque $B$. La probabilité que le centre du disque soit dans le carré de côté $(1-2.r)$ est proportionnelle à la surface de ce carré. D'où: $P=\frac{\textrm{surface petit carré}}{\textrm{surface grand carré}}=(1-2.r)^2$.

<img src="./Franc_Carreau_fig.01.svg" />

In [1]:
open Random;;
Random.self_init;;

- : unit -> unit = <fun>

In [2]:
let linspace start stop nbre =
    let step = (stop-.start)/.(float_of_int (nbre-1)) in
    let rec linspace num acc =
        if num=nbre then List.rev acc else
        linspace (num+1) ((start+.(float_of_int num)*.step)::acc) in
    linspace 0 [];;

val linspace : float -> float -> int -> float list = <fun>

In [3]:
#use "topfind";;
#require "plplot";;
open Plplot;;
module P = Plot;;
let couleurs_list = [[ 0;255;255;255]; (*`white*)
                     [ 1;  0;  0;  0]; (*`black*)
                     [ 2;  0;  0;255]; (*`blue*)
                     [ 3;255;  0;  0]; (*`red*)
                     [ 4;165; 42; 42]; (*`brown*)
                     [ 5;  0;  0;  0]; [ 6;  0;  0;  0]; [ 7;  0;  0;  0]; [ 8;  0;  0;  0]; [ 9;  0;  0;  0]; 
                     [10;200;200;200]; (*`gray*)
                     [11;  0;255;255]; (*`light_blue*)
                     [12;  0;255;  0]; (*`green*)
                     [13;255;255;  0]; (*`yellow*)
                     [14;255;  0;255]; (*`pink*)
                     [15;160;  0;213]; (*`purple*) ]
let rec loop couleurs_list = match couleurs_list with
    | [n;r;g;b]::tl -> plscol0 n r g b; loop tl
    | _ -> ();;
let couleurs = (fun () -> plscolbg 255 255 255; loop couleurs_list)
let initialisation filename xmin xmax ymin ymax = 
        P.init (xmin, ymin) (xmax, ymax) `greedy (`svg `core) ~filename:(filename^".svg") ~pre:couleurs
let xlabel texte = P.text_outside `black (`bottom 0.5) 3. texte
let ylabel texte = P.text_outside `black (`left 0.5) 5. texte 
let label texte_x texte_y titre = P.label texte_x texte_y titre

- : unit = ()
Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads

- : unit = ()


/usr/lib/ocaml/plplot: added to search path
/usr/lib/ocaml/plplot/plplot.cma: loaded


module P = Plplot.Plot

val couleurs_list : int list list =
 [[0; 255; 255; 255]; [1; 0; 0; 0]; [2; 0; 0; 255]; [3; 255; 0; 0];
 [4; 165; 42; 42]; [5; 0; 0; 0]; [6; 0; 0; 0]; [7; 0; 0; 0]; [8; 0; 0; 0];
 [9; 0; 0; 0]; [10; 200; 200; 200]; [11; 0; 255; 255]; [12; 0; 255; 0];
 [13; 255; 255; 0]; [14; 255; 0; 255]; [15; 160; 0; 213]]

val loop : int list list -> unit = <fun>

val couleurs : unit -> unit = <fun>

val initialisation : string -> float -> float -> float -> float -> P.stream_t =
 <fun>

val xlabel : string -> P.plot_t = <fun>

val ylabel : string -> P.plot_t = <fun>

val label : string -> string -> string -> P.plot_t = <fun>

In [4]:
let carreau r = 
    let x=Random.float 1. in
    let y=Random.float 1. in
    if x+.r>1. || x-.r<0. || y+.r>1. || y-.r<0. 
        then false
        else true;;
        
let simulation r nbre_simul =
    let rec loop num_simulation res =
       if num_simulation < nbre_simul 
          then if carreau r then loop (num_simulation+1) (res+1)
                            else loop (num_simulation+1) res
          else float_of_int res /. (float_of_int nbre_simul) in
    loop 0 0;;          

let calcul r =
    (1.-.2.*.r)**2.;;

val carreau : float -> bool = <fun>

val simulation : float -> int -> float = <fun>

val calcul : float -> float = <fun>

In [5]:
let liste_r=linspace 0. 0.5 100 in
let liste_proba=List.map (fun r -> simulation r (10*1000)) liste_r in
let liste_proba_th=List.map calcul liste_r in
let xs = Array.of_list liste_r in
let ys = Array.of_list liste_proba in
let ys' = Array.of_list liste_proba_th in

let p = initialisation "image" 0. 2. 0. 1. in
P.plot ~stream:p [P.points ~symbol:"+" `black xs ys; 
                  P.lines `red xs ys';
                  xlabel "a/l"; 
                  ylabel "probabilité";
                  P.legend [[P.symbol_legend "simulation" `black "+"];
                            [P.line_legend "théorie" `red]]];
P.finish ~stream:p ();;

- : unit = ()

<img src="./image.svg" width=750 />